# Homework on Panel Data

**Hint:**

**Use exercise notebook `An_introduction_to_Panel_Data.ipynb' in the Week 6 folder as a guide to complete this notebook.**

In [1]:
# !pip install panelsplit > /dev/null # installation of PanelSplit
# !pip install wbdata > /dev/null # installation of wbdata

In [ ]:
from panelsplit import PanelSplit
import wbdata
import pandas as pd; import numpy as np

Reload the example dataframe:

In [ ]:
import pandas as pd; import numpy as np

# Define indicators and countries of interest
indicators = {"NY.GDP.MKTP.CD": "GDP",  # Gross Domestic Product (current US$)
              "SP.POP.TOTL": "Population",  # Total population
              "FP.CPI.TOTL.ZG": "Inflation",     # Inflation, consumer prices (annual %)
              "NE.TRD.GNFS.ZS": "Trade_Balance"}    # Trade balance (% of GDP)


In [ ]:
# Select as countries following list: USA, Brazil, Russian Federation, India, China using the correct abbreviations.

countries = [
    
             # Fill out with your code! 
             
             ]

In [ ]:
features, target  = ['Population','Inflation','Trade_Balance'], 'GDP' # Define feature columns and target

# Fetch data from the World Bank API
df = wbdata.get_dataframe(indicators, country=countries).reset_index()
df['date'] = df.date.astype(int)

In [ ]:
# Choose as date range the years between 2003 (excludsing 2003) and 2023 (including 2023)

df = df.query(
              
                # Fill out with your code! 
              
              )

In [ ]:
print('a sample of the data:')
display(df.sample(5))

print('summary statistics:')
print(df.describe())

Say we don't have all the data available- 10% of the features are missing at random.

In [ ]:
# Select three different ratios of missing values for the three input features ['Population', 'Inflation', 'Trade_Balance']:
# Select the ratios 0.1, 0.12, 0.15, respectively.

df.loc[df.sample(
    
                # Fill out with your code! 
                 
                 ).index, features[0]] = np.nan

df.loc[df.sample(
    
                # Fill out with your code! 
                 
                 ).index, features[1]] = np.nan

df.loc[df.sample(
    
                # Fill out with your code! 
                 
                 ).index, features[2]] = np.nan

print('proportion of features that are missing:')
round(df[features].isna().mean(), 4)

We can use PanelSplit to apply imputation in a time-series fashion to fill in the missing data.

#### Initializing PanelSplit
- To initialize PanelSplit, we pass the time series to the periods argument.
- n_splits, gap, and test_size are all arguments used by [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) to split up the time series.
- When we specify `plot=True`, a graph is displayed. This is a helpful visualization to understand what this particular instance of PanelSplit is doing. By reading this graph, we see that there are 19 folds or splits, that range from 2003 up to 2023. Train sets are marked in blue and test sets are marked in red.

#### `cross_val_fit_transform`
- PanelSplit's cross_val_fit_transform takes any object that can implement fit/transform (like a scaler or imputer). 
- It iteratively fits on the train and transforms the test for each fold. 
- By setting `include_test_in_fit = True`, we also include the test data during fitting for each fold.


In [ ]:
# Choose the appropriate split for the above date range of 2003 - 2023:

n_splits =          # Fill out with your code! 

panel_split = PanelSplit(periods=df.date, n_splits = n_splits, gap=0, test_size=1, plot=True)

In [ ]:
# Use a different imputation technique: Use SimpleImputer(missing_values=np.nan, strategy='mean')

from sklearn.impute import SimpleImputer

df[features], fitted_imputers = panel_split.cross_val_fit_transform(
    
    # Fill out with your code! 
    
    )

The only observations which are still missing are in 2004. This is because that is the train set for first split.

In [ ]:
df[df.isna().any(axis=1)]

We can go ahead and implement imputation for observations that take place in January of 2001

In [ ]:
# Perform the imputation for year 2004 and make sure that there are no remaining missing values:

df.loc[df.date == 2004, features] =         # Fill out with your code! 

df.isna().sum()

Now that all observations have all features, we can determine the best set of hyper-parameters to use.

#### Hyper-parameter search

Since PanelSplit is compatible with sklearn, we can use any sklearn hyper-parameter optimizer has takes cv as an argument.

Let's say that our validation period is 2013-2022. Exclude the data for year 2023 from the validationn data set


In [ ]:
# As model choose the XGBoost regressor.

# Choose a reasonable parameter grid for the 3 hyperparameters 'max_depth', 'min_child_weight', 'max_leaves'.

# Select all data up to excluding the year 2023 as validation data.

# Select 10 splits. 

# Perform your hyperparameter tuning via grid search to get the best set of hyperparameters.

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

validation_df = df.query('date < 2023')

param_grid =            # Fill out with your code! 

panel_split = PanelSplit(validation_df.date, n_splits=10, gap=0, test_size=1, plot=True)

grid_search = GridSearchCV(
    
    # Fill out with your code! 
    
    )

grid_search.fit(validation_df[features], validation_df[target])

print('GridSearch results:')
display(pd.DataFrame(grid_search.cv_results_))

In [ ]:
# Return the best fit parameters:

best_params =           # Fill out with your code! 

print(best_params)

Now that we've determined what hyper-parameters to use, let's generate predictions in the test period.

#### `cross_val_fit_predict`
- PanelSplit's cross_val_fit_predict iteratively fits on the train and predicts the test for each fold
- Here are the arguments I use:
    - estimator: estimator object.
    - X: Features.
    - y: Target variable.
    - labels: pandas DataFrame containing labels for the target variable predicted by the model. The predicted target will be a new column added to this DataFrame.
    - prediction_method: The prediction method to use. It can be 'predict', 'predict_proba', or 'predict_log_proba'. Default is 'predict'.
    - y_pred_col: Column name for the predicted values. Default is None, in which case the name will be the name of y.name + 'pred'. If y does not have a name attribute, the name will be 'y_pred'.
    - n_jobs: The number of jobs to run in parallel

In [ ]:
# Choose 10 splits.

# Perform XGBoost regression prediction via cross validation.

# Note: For `panel_split.cross_val_fit_predict(...)` use the complete dataframe `df`.

panel_split = PanelSplit(
    
    # Fill out with your code! 
    
    )

result_df = panel_split.gen_test_labels(df[['country','date', 'GDP']])

result_df['GDP_pred'], models = panel_split.cross_val_fit_predict(
   
                                                                   # Fill out with your code! 
                                                                      
                                                                  )

The output of cross_val_fit_predict is a dataframe of predictions and fitted models. This is what the DataFrame looks like:

In [ ]:
result_df

Let's assess performance:

In [ ]:
# Use three different metrics - RMSE (root mean squared), MAE (mean absolute error), MAPE (mean absolute percentage error) 
# to evaluate your model performance:

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

print('RMSE:', 
        ...     # Fill out with your code! 
      )

print('MAE:', 
        ...     # Fill out with your code! 
      )

print('MAPE:', 
        ...     # Fill out with your code!                   
      )

## Multiple Snapshots/Updated Values

There are also instances where you might want to keep observations seperate between folds, for example when scaling features. 

In this case, it is recommended that you use `cross_val_fit_predict` with a Pipeline object.

However, in case you need to manually implement something where observations must be handled differently across folds, PanelSplit's `gen_snapshots` converts a single df to multiple snapshots of that dataframe.

In [ ]:
# Create snapshots of the data using n_splits = (2023 - 2013):

panel_split = PanelSplit(      
                         
                         # Fill out with your code! 
                         
                         )

In [ ]:
snapshot_df = panel_split.gen_snapshots(df, period_col = 'date')
print('2 new columns are created: split and snapshot_period:')
display(snapshot_df)
print(f'before gen_snapshot:{df.shape} after gen_snapshot:{snapshot_df.shape}')

Let's scale our data using `cross_val_fit_transform`. Here each snapshot is scaled separately. We also specify `transform_train=True`, thereby also transforming the training set for each fold.

In [ ]:
panel_split = PanelSplit(periods = snapshot_df.date, snapshots = snapshot_df.snapshot_period, n_splits = (2023 - 2013)) # notice the usage of the snapshots argument

In [ ]:
# Apply MinMaxScaler to rescale the data:
from sklearn.preprocessing import MinMaxScaler

snapshot_df[features], scalers = panel_split.cross_val_fit_transform(
    
        # Fill out with your code! 
    
        ) # transform_train transforms the training set for each fold

In [ ]:
display(snapshot_df)

# Submission Instructions:

- Go to the **Assessments** tab in iCollege and click on **Assignments.** Submit your solution under the **Homework 3** category.

- Report your values for **RMSE, MAE and MAPE** for your model performnace in the Homework submission field in iCollege.

- Attach this **executed (!)** Jupyter notebook and submit with your response above in iCollege.